In [1]:
import datetime
import requests
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr,norm
from sklearn.linear_model import LinearRegression,Ridge,RidgeCV,Lasso
from sklearn.svm import LinearSVR,SVR
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error,mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
import pyreadr
from statsmodels.tsa.seasonal import STL

In [2]:
work_path=r'D:\university\研究生\tourism_forecasting_competition\submit\code and supplementary materials'##work_path must be adjusted before running the code.
os.chdir(work_path)

In [3]:
country_name=["加拿大",'智利','墨西哥','台湾','香港','日本','韩国','澳门','马尔代夫','柬埔寨','印尼','新加坡','新西兰','美国','泰国','土耳其','澳大利亚','夏威夷','奥地利','捷克']
reference=pd.read_excel(r'reference.xlsx').iloc[:,1:]
data=pd.read_excel(r'data.xlsx').iloc[:,1:]
data.columns=country_name
data.index=pd.date_range(start="1989-1", end="2023-02", freq="MS")
data_full=data.iloc[:372,:]
data_pre=data.iloc[360:372,:]
data_act=data.iloc[371:408,:]
logistic_prediction=pd.read_csv('logistic_forecast.csv',encoding='gbk')

In [5]:
def get_linear_equation(p1,p2):
    x1,x2=p1[0],p2[0]
    y1,y2=p1[1],p2[1]
    slope=(y1-y2)/(x1-x2)
    intercept=y2-slope*x2
    return (slope,intercept)

In [6]:
recovery_intensity=[0.7,0.7,1,0.6,0.85,0.8,0.8,0.85,0.8,0.8,0.8,0.8,0.65,0.65,0.85,0.75,0.8,0.8,0.65,0.65]   ###the recover intensity we estimate
baseline_pre=pd.read_excel('baseline.xlsx').iloc[:,1:]

In [7]:
quadratic_prediction=pd.DataFrame(columns=country_name,index=pd.date_range(start="2023-08", end="2024-07", freq="MS"))
for i in range(20):
    ##calculate the sesonality
    stl = STL(np.log(data_full.dropna().iloc[:,i]), period=12, seasonal=13)
    res = stl.fit()
    seasonal=pd.DataFrame(res.seasonal[:-1])
    q=pd.Series(seasonal.index).apply(lambda x:x.month)
    seasonal['month']=np.array(q)
    seasonal_mean=seasonal.groupby(by='month').mean()

    ###construct baseline and reference series
    t1=data_pre.iloc[:,i]
    t2=baseline_pre.iloc[:,i]
    t3=t2.copy()
    t4=t2[:12]
    baseline=pd.concat([t1,t3,t4,t2[:19]],axis=0)
    baseline.index=pd.date_range(start="2019-1", end="2024-7", freq="MS")
    t5=data_act.iloc[:,i]
    t6=pd.Series(reference.iloc[:,i])
    adjust=pd.concat([t5,t6],axis=0)
    adjust.index=pd.date_range(start="2019-12", end="2023-6", freq="MS")

    ###quadratic curve fitting
    x=np.arange(18)
    y=np.array(adjust[25:])
    for m,n in enumerate(list(adjust[25:].index.month-1)[1:]):
        y[m]=y[m]*(1+seasonal_mean.iloc[n,0])
    weight=np.ones(18)
    x=np.append(x,31)
    y=np.append(y,(baseline[-1]/(1+seasonal_mean.iloc[6,0]))*recovery_intensity[i])
    weight=np.append(weight,18)
    x_pre=np.arange(31)
    coef=np.polyfit(x,y,w=weight,deg=2)
    y_fit=np.polyval(coef,x_pre)
    y_fit=pd.Series(y_fit)
    y_fit.index=pd.date_range(start="2022-1", end="2024-7", freq="MS")
    y_fit=y_fit[17:]
    y_fit[0]=adjust[-1]
    
    ##multiply the seasonality back
    for m,n in enumerate([6,7,8,9,10,11,0,1,2,3,4,5,6]):
        y_fit[m+1]=y_fit[m+1]*(1+seasonal_mean.iloc[n,0])
        quadratic_prediction[country_name[i]]=y_fit[2:]

In [23]:
linear_prediction=pd.DataFrame(columns=country_name,index=pd.date_range(start="2023-08", end="2024-07", freq="MS"))
for i in range(20):
    ##calculate the sesonality
    stl = STL(np.log(data_full.dropna().iloc[:,i]), period=12, seasonal=13)
    res = stl.fit()
    seasonal=pd.DataFrame(res.seasonal[:-1])
    q=pd.Series(seasonal.index).apply(lambda x:x.month)
    seasonal['month']=np.array(q)
    seasonal_mean=seasonal.groupby(by='month').mean()

    ###construct baseline and reference series
    t1=data_pre.iloc[:,i]
    t2=baseline_pre.iloc[:,i]
    t3=t2.copy()
    t4=t2[:12]
    baseline=pd.concat([t1,t3,t4,t2[:19]],axis=0)
    baseline.index=pd.date_range(start="2019-1", end="2024-7", freq="MS")
    t5=data_act.iloc[:,i]
    t6=pd.Series(reference.iloc[:,i])
    adjust=pd.concat([t5,t6],axis=0)
    adjust.index=pd.date_range(start="2019-12", end="2023-6", freq="MS")

    ###linear linkage
    start=(1,adjust[-1]/(1+seasonal_mean.iloc[5,0]))
    end=(14,(baseline[-1]/(1+seasonal_mean.iloc[6,0]))*recovery_intensity[i])
    coef=get_linear_equation(start,end)
    y_fit=[x*coef[0]+coef[1] for x in range(-16,15)]
    y_fit=pd.Series(y_fit)
    y_fit.index=pd.date_range(start="2022-1", end="2024-7", freq="MS")
    y_fit=y_fit[17:]
    y_fit[0]=adjust[-1]

    ##multiply the seasonality back
    for m,n in enumerate([6,7,8,9,10,11,0,1,2,3,4,5,6]):
        y_fit[m+1]=y_fit[m+1]*(1+seasonal_mean.iloc[n,0])
        linear_prediction[country_name[i]]=y_fit[2:]

In [9]:
reference

,加拿大,智利,墨西哥,台湾,香港,日本,韩国,澳门,马尔代夫,柬埔寨,印尼,新加坡,新西兰,美国,泰国,土耳其,澳大利亚,夏威夷,奥地利,捷克
0,7519.212331,1014.857143,12395.571429,9033.571429,219587.571429,31538.142857,25291.857143,8.980627e+05,5320.571429,23727.857143,27446.000000,27123.142857,4917.857143,61250.142857,86456.000000,9192.571429,14522.857143,1422.285714,4829.428571,7995.866961
1,9281.011587,1163.819352,9966.062799,11907.656261,114632.142857,35485.714286,35867.939572,1.163862e+06,20352.838426,38980.299100,39707.600708,34309.000000,7952.404845,44051.428571,205795.992098,9628.254008,55833.042069,1807.104679,3830.872334,12064.820285
2,7482.356930,1331.264804,14027.000042,13938.104484,387599.508738,43107.809095,45991.222223,1.297000e+06,17712.004638,45107.818815,45078.990425,71132.630839,7960.235677,50203.964166,314368.226043,12452.005017,48120.197711,1540.626060,8669.252308,15598.556530
3,8607.643894,1140.485942,11392.725189,17896.754445,429153.268331,54694.706693,79779.775446,1.356445e+06,23249.162255,42612.150675,51408.340651,91219.643868,8722.064078,47723.837651,420656.549385,17643.460538,49074.523250,1653.653346,12439.820192,14944.177564
4,12577.574204,1229.509641,12906.002658,18337.194671,508270.617828,62734.571831,110415.854784,1.363906e+06,35758.662465,35563.271094,63766.488734,89358.283661,8279.791641,62209.311530,436496.695254,22752.759622,40634.406026,2702.055537,15060.832598,18174.445426
5,14458.020634,1091.659271,11934.017864,17316.967843,464714.313869,71023.594939,114424.598939,1.114992e+06,42767.107890,31620.085076,136943.189087,94463.355824,6168.699701,67545.486098,425597.601276,22920.836404,41491.905792,2900.487297,15119.007160,16818.131004


In [15]:
real=pd.read_excel('real_value.xlsx')

In [50]:
final_prediction=pd.read_excel('point_forecast.xlsx').set_index('Unnamed: 0')

In [51]:
real_value=real.iloc[1:,1:]
real_value.index=final_prediction.index[:6]

In [69]:
((real_value-final_prediction/recovery_intensity).abs().mean()/real.iloc[:,1:].diff().abs().mean()).to_excel('error1.xlsx')

In [67]:
final_prediction/recovery_intensity

,加拿大,智利,墨西哥,台湾,香港,日本,韩国,澳门,马尔代夫,柬埔寨,印尼,新加坡,新西兰,美国,泰国,土耳其,澳大利亚,夏威夷,奥地利,捷克
Unnamed: 0,,,,,,,,,,,,,,,,,,,,
2023-08-01,40347.427048,1639.591676,11180.857465,51197.279815,1.163738e+06,282855.290125,244440.446746,1.631293e+06,36436.809306,75966.361409,144318.598188,185370.137948,19326.838007,143486.380096,570291.971164,23032.220840,67742.779153,3355.880460,40281.569994,47151.195224
2023-09-01,37840.351427,1542.403889,10814.217532,50767.926837,1.016657e+06,290557.080036,247664.722294,1.349580e+06,31211.429146,72986.032794,130506.321476,128971.956598,18448.226868,124801.553798,485257.502765,26880.057539,61180.757531,3272.819967,44289.302534,44767.454014
2023-10-01,29468.929926,2273.755294,11183.310781,61056.703986,1.266602e+06,314353.326524,276485.954802,1.661503e+06,31196.650012,87862.784277,126512.366371,144780.874351,19178.733228,104457.583268,483364.117517,26216.771276,64106.547795,3210.406555,49683.378567,49694.192963
2023-11-01,15521.692244,2740.464649,15358.655829,66855.567671,1.318008e+06,331311.023740,251586.123869,1.721225e+06,21968.309487,98640.924013,113386.695981,157493.331232,22214.024606,97860.630055,530716.246113,23953.715203,76573.461747,2890.245064,36308.217741,28414.343089
2023-12-01,25525.195587,2838.566851,21480.798090,74992.193117,1.633318e+06,296083.361242,269640.045783,1.797864e+06,16093.254267,115367.653486,102258.505908,172464.480179,20845.106623,121331.525497,570927.818257,21582.720845,85856.465153,3589.043342,32160.871755,26577.196494
2024-01-01,27901.324055,2531.609004,11969.177730,83569.302733,1.849370e+06,463607.110570,290249.619166,1.919908e+06,25536.527111,127852.771644,154451.912876,229475.946130,26100.624714,192577.386624,735218.671050,19997.529659,106375.178768,4411.087478,19992.066882,13863.366298
2024-02-01,29787.961953,2618.321641,12978.268480,102247.634486,1.748286e+06,507161.824707,367469.835626,2.025322e+06,32049.694034,161403.458206,212064.544952,280801.345562,32109.621193,172757.329320,853959.961332,25104.174651,143539.574522,4825.201869,32328.926231,27606.575838
2024-03-01,21599.947126,3040.340585,20553.898157,112447.036137,1.724099e+06,580309.166743,401769.394229,1.948527e+06,16578.455282,152790.540084,158335.810590,243199.928718,28611.323614,148367.019734,777542.790690,27843.214328,100372.597108,3888.454338,37150.848571,34885.284854
2024-04-01,31159.242949,2841.763821,12058.131373,139385.959572,1.886654e+06,646126.088458,449662.825815,2.071973e+06,18190.052538,140041.771698,163173.626737,260659.780936,30721.090554,143656.434715,818944.525863,30399.809719,94935.413289,4553.103888,56097.454409,52172.391322


In [13]:
logistic_prediction.index=pd.date_range(start='2023-08',end='2024-07',freq='MS')
logistic_prediction.columns=country_name

In [43]:
logistic_prediction

,加拿大,智利,墨西哥,台湾,香港,日本,韩国,澳门,马尔代夫,柬埔寨,印尼,新加坡,新西兰,美国,泰国,土耳其,澳大利亚,夏威夷,奥地利,捷克
2023-08-01,33372.64131,987.598072,8339.988295,35496.05222,1643675.185,242749.2631,182310.1184,1255828.535,12283.753560,72916.83603,82674.74938,145834.3590,15831.352700,62612.15194,425743.9757,11083.00512,42252.72135,2847.847554,23466.45232,34436.20131
2023-09-01,29765.69521,862.825615,8441.320523,36835.57997,1398130.960,247821.3504,177715.5522,1118316.970,11999.115820,71681.01975,74882.30882,105143.2720,12506.849220,65953.27962,364199.8240,12023.95901,41178.49401,2484.621836,32285.93400,30876.09603
2023-10-01,23080.01210,1326.263645,8801.847251,46494.82948,1753060.441,263139.8445,201640.8400,1433425.552,15629.819670,84796.95378,75092.22248,126727.2335,14005.497820,73420.14437,396928.3959,14405.14716,49307.42946,3264.023636,34490.45954,34022.85872
2023-11-01,16606.40454,1674.099198,11557.137100,54524.08473,1857442.356,247322.3006,180867.0379,1574534.783,9949.004428,93174.31603,71793.43607,145155.5279,10590.010180,94947.90628,471063.3909,17054.74648,63973.74138,4086.331288,32154.24536,19106.99449
2023-12-01,19651.43374,1708.910598,16096.123120,57069.75714,2277985.474,218812.5564,193146.7896,1680180.270,9815.099216,108270.15650,65151.53759,153119.9959,8079.604239,101546.01010,498915.6199,16198.34205,73407.21987,4339.029713,34245.21094,18494.22906
2024-01-01,19895.80321,1530.862726,9297.940883,59836.07053,2539653.711,355599.5563,209153.5520,1806037.130,14954.460020,123711.41230,102054.68040,191040.0068,12267.609470,139532.52600,619610.0803,17819.90424,83629.79888,4844.585586,25167.15624,13625.61354
2024-02-01,19313.48688,1598.788166,10071.287430,73696.04206,2397757.511,350199.0468,262454.4516,1867852.139,19862.287820,154316.89630,143841.41650,229340.7636,13960.244200,170924.10830,704889.2702,18398.20613,107095.85260,4601.161877,20576.05155,19487.54168
2024-03-01,18090.57774,1926.218486,16283.882720,89917.06860,2375449.830,424246.3784,271352.5519,1928139.794,9599.654529,131817.73970,104364.73420,196190.5296,14979.903300,139749.25550,614678.6293,21694.16127,74924.38337,5730.845053,15654.89977,23001.00236
2024-04-01,22469.20220,1849.417927,10862.639990,115357.07550,2496245.030,474577.2352,306571.2699,1947074.365,12015.945010,129832.51490,109012.10050,208915.9701,16661.199210,117009.03160,665203.9939,21132.53180,76856.83079,5469.207141,20007.36100,32156.91372
2024-05-01,31483.90560,1559.911700,10855.743390,109973.18980,2586182.955,413566.8942,320344.7626,2005456.137,16449.850300,114385.78450,106499.79970,177661.3985,25204.723740,106634.92800,593418.4491,19350.48415,69492.40032,5559.579492,23512.09717,47892.36346


In [24]:
final_prediction=(linear_prediction+quadratic_prediction+logistic_prediction)/3
# final_prediction.to_excel('point_forecast.xlsx')

In [44]:
final_prediction

,加拿大,智利,墨西哥,台湾,香港,日本,韩国,澳门,马尔代夫,柬埔寨,印尼,新加坡,新西兰,美国,泰国,土耳其,澳大利亚,夏威夷,奥地利,捷克
2023-08-01,28243.198933,1147.714173,11180.857465,30718.367889,9.891774e+05,226284.232100,195552.357396,1.386599e+06,29149.447445,60773.089127,115454.878551,148296.110359,12562.444705,93266.147063,484748.175490,17274.165630,54194.223323,2684.704368,26183.020496,30648.276896
2023-09-01,26488.245999,1079.682722,10814.217532,30460.756102,8.641583e+05,232445.664029,198131.777835,1.147143e+06,24969.143317,58388.826235,104405.057181,103177.565278,11991.347464,81121.009969,412468.877351,20160.043154,48944.606025,2618.255974,28788.046647,29098.845109
2023-10-01,20628.250948,1591.628706,11183.310781,36634.022392,1.076612e+06,251482.661219,221188.763842,1.412277e+06,24957.320010,70290.227422,101209.893097,115824.699480,12466.176598,67897.429124,410859.499889,19662.578457,51285.238236,2568.325244,32294.196068,32301.225426
2023-11-01,10865.184571,1918.325254,15358.655829,40113.340603,1.120307e+06,265048.818992,201268.899095,1.463041e+06,17574.647589,78912.739211,90709.356785,125994.664985,14439.115994,63609.409536,451108.809196,17965.286402,61258.769398,2312.196052,23600.341531,18469.323008
2023-12-01,17867.636911,1986.996795,21480.798090,44995.315870,1.388320e+06,236866.688994,215712.036627,1.528184e+06,12874.603413,92294.122789,81806.804727,137971.584143,13549.319305,78865.491573,485288.645518,16187.040634,68685.172122,2871.234674,20904.566641,17275.177721
2024-01-01,19530.926838,1772.126302,11969.177730,50141.581640,1.571965e+06,370885.688456,232199.695332,1.631922e+06,20429.221689,102282.217315,123561.530301,183580.756904,16965.406064,125175.301306,624935.870393,14998.147245,85100.143014,3528.869983,12994.843473,9011.188094
2024-02-01,20851.573367,1832.825148,12978.268480,61348.580692,1.486043e+06,405729.459766,293975.868501,1.721523e+06,25639.755227,129122.766565,169651.635962,224641.076449,20871.253776,112292.264058,725865.967132,18828.130988,114831.659617,3860.161495,21013.802050,17944.274294
2024-03-01,15119.962988,2128.238409,20553.898157,67468.221682,1.465484e+06,464247.333395,321415.515383,1.656248e+06,13262.764226,122232.432067,126668.648472,194559.942974,18597.360349,96438.562827,660911.372087,20882.410746,80298.077686,3110.763471,24148.051571,22675.435155
2024-04-01,21811.470064,1989.234675,12058.131373,83631.575743,1.603656e+06,516900.870766,359730.260652,1.761177e+06,14552.042030,112033.417358,130538.901389,208527.824749,19968.708860,93376.682564,696102.846983,22799.857289,75948.330631,3642.483111,36463.345366,33912.054359
2024-05-01,34598.249298,1684.584061,12729.845045,80837.832921,1.681936e+06,505476.320153,378375.226658,1.789670e+06,17189.059682,102415.172647,125968.824607,188026.718273,19011.310115,112883.854494,648370.391683,24063.113590,63958.568052,4211.293150,46322.773111,49951.854801
